In [166]:
import requests
import urllib, json
import datetime
import pandas as pd

In [127]:
class Limit_call:
    def __init__(self,f, cmax = 120, time = 60):
        self._f = f
        self.counter = 0
        self.cycle = time
        self.counter_max = cmax
        
    def __call__(self,*args,**kargs):
        if self.counter == 0:
            self.start = datetime.datetime.now()
        if datetime.datetime.now() - self.start > datetime.timedelta(seconds = self.cycle):
            self.counter = 0 #Reset after a cycle 
            self.start = datetime.datetime.now()
        if self.counter+1 >= self.counter_max:
            remain = (self.start + datetime.timedelta(seconds = self.cycle) - datetime.datetime.now()).seconds
            raise Exception(f"Calling Celiling reached. Counter will RESET in {remain} seconds")
        else:
            self.counter += 1 
            return self._f(*args,**kargs)
            
@Limit_call
def Request(*args,**kargs):
    return requests.request(*args,**kargs)

In [50]:
class MTK:
    def __init__(self,token = None, project_id = None,info_show = True):
        """
        Initialize the class with Token obtained @ 
        https://developers.meistertask.com/docs/authentication
        """
        self.set_token(token)
        self.set_project_id(project_id)
        if info_show:
            self.info()
        
    def set_token(self,token):
        self.__token = token 
        self.headers_get =  {'Accept' : '*/*',
          'Authorization': f'Bearer {self.__token}',
           'accept-encoding' : 'gzip, deflate'}
        self.headers_post = dict(self.headers_get)
        self.headers_post ['content-type'] = 'application/json'
        self.headers_post ['content-length'] = '69'
        
    def set_project_id(self,ID):
        self.project_id = ID
        
    def _QUERY(self,url,params = {},headers = None):
        """
        General Query Function.Deafult GET header 
        """
        if headers is None:
            return Request(method = "GET", url = url,
                                headers = self.headers_get,params = params).json()
        else:
            return Request(method = "GET", url = url,
                                headers = headers,params = params).json()
        
    def Get_info(self):
        self.PROJECTs = {d['id']:d['name'] for d in self._QUERY("https://www.meistertask.com/api/projects")}
        if self.project_id is not None:
            self.SECTIONs = {d['id']:d['name'] for d in 
                         self._QUERY(f"https://www.meistertask.com/api/projects/{self.project_id}/sections")}
        
    def info(self):
        self.Get_info()
        try:
            print(f"[Projects]:")
            print("   ID   |Name")
            for ID,name in self.PROJECTs.items():
                print(f"{ID}|{name}")
            print(f"[Sections under Proejct {self.PROJECTs[str(self.project_id)]}]")
            print("   ID   |Name")
            for ID,name in self.SECTIONs.items():
                print(f"{ID}|{name}")
        except KeyError:
            pass 
        
    def Get_task(self,task_id = "",project_id = None,params={}):
        """
        [Params]
        @project_id: id of the project of which the task belongs to
        @id: the id of the task. If blank, return all avaliable tasks
        ### https://developers.meistertask.com/docs/get-task
        [Query params]:
        @assigned_to_me:	string
            If set to true, 1 or yes, returns only tasks that are assigned to the current person.
        @focused_by_me:	string
            If set to true, 1 or yes, returns only tasks that are focused by the current person.
        @status:	string
            Filters the tasks based on their status. Any of: open, completed, completed_archived, trashed.
        @items:	integer
            Controls how many items should be shown on each page.
        @page:	integer
            Controls which page to view.
        @sort:	string
            Controls how to sort results.
        """
        try:
            if project_id is not None:
                return self._QUERY(url = f"https://www.meistertask.com/api/projects/{project_id}/tasks/{task_id}",
                                  params = params)
            else:
                return self._QUERY(url = f"https://www.meistertask.com/api/projects/{self.project_id}/tasks/{task_id}",
                                  params = params)
        except:
            return self._QUERY(url = f"https://www.meistertask.com/api/tasks/{task_id}",
                                  params = params)
    
    def Post_task(self,section_id,name:str,notes:str = "",
                  label_ids:[str] = [],custom_fields:[{str:str}] = [],
                 checklists:[{str:str}] = []):
        """
        [Params] (Requred)
        @section_id: the id of the section of which the task is posted under 
        @name: name of the task to be posted 
        ###https://developers.meistertask.com/docs/post-task
        [Body Params] (Optional)
        """
        data = {"name": name,"notes": notes,
                "label_ids": label_ids, "custom_fields": custom_fields, "checklists": checklists} 
        data = json.dumps(data, indent = 4)  
        return Request(method = "POST", url = f"https://www.meistertask.com/api/sections/{section_id}/tasks",
                                headers = self.headers_post ,params = {},data = data).json()
    
    def Post_project(self,name,description):
        data = json.dumps({"name":name,"notes" : description},indent = 4)
        return Request(method = "POST", url = "https://www.meistertask.com/api/projects",
                                headers = self.headers_post ,params = {},data = data).json()
    
    def Post_section(self,name,project_id = None):
        if project_id is None:
            project_id = self.project_id
        data = json.dumps({"name":name},indent = 4)
        header = dict(self.headers_post)
        header['content-length'] = '21'
        return Request(method = "POST", url = f"https://www.meistertask.com/api/projects/{project_id}/sections",
                                headers = header ,params = {},data = data).json()
 
        

In [188]:
class GPK_MTK(MTK):
    def __init__(self,token = None,project_id=None):
        self.SEC = {}
        MTK.__init__(self,token = token, project_id = None ,info_show = False)
        self.Get_info() #Collect Info about the account 
        if "__Created" not in self.__dict__ and project_id is None:
            print("CREATING PROJECT")
            self.RESET()
        else:
            self.project_id = project_id
        
    def RESET(self,sections = ['Upcoming','In Progress','Done Today','Personal Dev',
                              'Carrer','Health','Family','Other']):
        """
        Create a New Project with specified Sections 
        """
        self.project_info = self.Post_project("MTK_OKR","Objective and KeyResults Management")
        print("Project Initialized")
        self.set_project_id(self.project_info['id'])
        for sec in sections:
            section = MTK.Post_section(self,sec,self.project_id)
            print(f"Section {sec} Initialized")
            self.SEC[sec] = []
        print("RESET Complete")
        self.__Created = True
        
    def Sync(self):
        self.SEC = {}
        Tasks = self.Get_task(params = {"status":'open'})
        for task in Tasks:
            sec =  task['section_name']
            if sec in self.SEC:
                self.SEC[sec].append (task)
            else:
                self.SEC[sec] = [task]
                
    def View_df(self,section):
        TEMP = {}
        for feature in self.SEC[section][0].keys():
            TEMP[feature] = []
            for task in self.SEC[section]:
                TEMP[feature].append(task[feature])
        return pd.DataFrame(TEMP)
    

In [189]:
#TST = MTK('u1IqrMvqjFA99sNL9_RnipaYnXKd9cc7wUZXHCUhJ-I')

In [190]:
TEST = GPK_MTK('u1IqrMvqjFA99sNL9_RnipaYnXKd9cc7wUZXHCUhJ-I','4963813')
TEST.Sync()
TEST.SEC.keys()

In [196]:
TEST.View_df('Planed_Today')

,id,token,name,notes,notes_html,status,status_updated_at,section_id,section_name,project_id,sequence,assigned_to_id,assignee_name,tracked_time,due,created_at,updated_at
0,97251571,o7b2ob22,Fix technical issues and start working,None,,1,2021-03-21T02:19:13.711228Z,18891013,Planed_Today,4963813,52500.0,40597705,,0,None,2021-03-21T02:19:13.711426Z,2021-06-14T07:10:16.853180Z
1,97809943,6yDsheKH,REBOOT OKR,None,,1,2021-03-29T01:00:06.878813Z,18891013,Planed_Today,4963813,67500.0,40597705,,0,None,2021-03-29T01:00:06.879082Z,2021-06-14T07:10:19.383596Z
2,101329774,40MU5JUM,Finish Self-Evaluation at PrincipleU,None,,1,2021-05-15T14:09:30.385270Z,18891013,Planed_Today,4963813,37500.0,40597705,,0,None,2021-05-15T14:09:30.385657Z,2021-06-14T07:10:09.812881Z
3,103335319,lP82ZnLm,Strategy1: Sleep by 11P.M; Wake by 8 A.M,None,,1,2021-06-11T19:56:04.066886Z,18891013,Planed_Today,4963813,-15000.0,40597705,,0,None,2021-06-11T19:56:04.067077Z,2021-06-14T07:09:56.401147Z
4,103335330,y1c1JEoi,Strat 5: Reading for 1 Hour,None,,1,2021-06-11T19:56:05.714477Z,18891013,Planed_Today,4963813,22500.0,40597705,,0,None,2021-06-11T19:56:05.714703Z,2021-06-14T07:10:01.056262Z
5,103335331,8TvVZMOh,Stragegy2: No TV/Video Game during Lunch/ Dinn...,None,,1,2021-06-11T19:56:05.764257Z,18891013,Planed_Today,4963813,-7500.0,40597705,,0,None,2021-06-11T19:56:05.764554Z,2021-06-14T07:10:00.048383Z
6,103335337,e1TTXGkA,Strategy 3: Focus on doing one thing only,None,,1,2021-06-11T19:56:05.816633Z,18891013,Planed_Today,4963813,7500.0,40597705,,0,None,2021-06-11T19:56:05.816872Z,2021-06-14T07:09:58.921292Z
7,103335341,zMJvUOck,Strategy 4: Exercise (Walk 6000 Steps or 30 mi...,None,,1,2021-06-11T19:56:05.867579Z,18891013,Planed_Today,4963813,15000.0,40597705,,0,None,2021-06-11T19:56:05.867792Z,2021-06-14T07:10:05.784572Z
